## Paginación en Amazon DynamoDB

La paginación es el proceso de enviar solicitudes posteriores para continuar cuando una solicitud anterior está incompleta. Una operación Query o Scan en DynamoDB puede devolver resultados que están incompletos y requieren solicitudes posteriores para obtener el conjunto de resultados completo. Esto se debe a que DynamoDB pagina los resultados de una operación Query o Scan y devuelve un máximo de 1 MB de datos en una sola operación. Este es un límite estricto en DynamoDB. Con la paginación, los resultados de las operaciones de análisis y consulta se dividen en páginas de datos que tienen un tamaño de 1 MB o menos.

Para paginar los resultados y recuperarlos página a página, compruebe el resultado de bajo nivel de una operación Scan o Query para ver si el resultado contiene el elemento `LastEvaluatedKey`. Puede obtener los resultados restantes de la solicitud inicial mediante otra operación Scan o Query con los mismos parámetros, pero utilizando el valor `LastEvaluatedKey` como el parámetro `ExclusiveStartKey`. Si el resultado no incluye el valor LastEvaluatedKey, no hay elementos que recuperar.

In [1]:
import boto3
from botocore.exceptions import ClientError
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime
from ksuid import Ksuid
import time
import random
from pprint import pprint

In [2]:
dt=DynamoTable()
try:
    dt.select_table('FacebookTable')
    print(dt)
except:
    dt.create_table(
        table_name='FacebookTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S'
    )

- Table name: FacebookTable            
- Table arn: arn:aws:dynamodb:us-east-1:999470467758:table/FacebookTable            
- Table creation: 2023-07-06 13:20:53            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


### Añadir un nuevo tipo de entidad a una colección de elementos existente

En este escenario, puede intentar reutilizar una colección de elementos existente. Esto es genial si tu entidad padre está en una colección de ítems que no está siendo usada para una relación existente.

Por ejemplo, piense en una aplicación como Facebook antes de que introdujera el botón "Me gusta". Es posible que su tabla DynamoDB tuviera elementos Post que representaran una publicación concreta realizada por un usuario.

In [73]:
count = 500
names = ['Alex DeBrie', 'James Dean', 'Laura Dern']
all_items = []
pk = []

# Generate random date 
start_date = datetime(2019, 1, 1, 00, 00, 00)
end_date = datetime(2023, 12, 31, 23, 59, 59)
rand_date = [start_date + (end_date - start_date) * random.random() for i in range(int(1400))]

for i in range(int(count * 0.6)):
    ksuid = Ksuid()
    name = random.choice(names)
    Item={
            'PK': 'POST#' + str(ksuid),
            'SK': 'POST#' + str(ksuid),
            'PostOwner': name,
            'PostContent': 'https://...',
            'PostData': random.choice(rand_date).strftime("%Y-%m-%d %H:%M:%S"),
            'Type': 'POST'
    }
    all_items.append(Item)
    pk.append('POST#' + str(ksuid))

Entonces alguien tiene la brillante idea de permitir a los usuarios "Me gusta" a través del botón Me gusta. Cuando queremos añadir esta funcionalidad, tenemos un patrón de acceso de "Obtener la entrada y los "Me gusta" de la entrada".

En esta situación, la entidad Like es un tipo de entidad completamente nuevo, y queremos obtenerla al mismo tiempo que obtenemos la entidad Post. Si miramos nuestra tabla base, la colección de ítems para la entidad Post no está siendo utilizada para nada. Podemos añadir nuestros ítems Like a esa colección utilizando el siguiente patrón de clave primaria para Likes:

In [74]:
pk = pk[0:10]
for i in range(int(count * 0.2)):
    ksuid = Ksuid()
    Item={
        'PK': random.choice(pk),
        'SK': 'LIKE#USERID#' + str(ksuid),
        'LikingUser': name,
        'Type': 'LIKE'
    }
    all_items.append(Item)

for i in range(int(count * 0.2)):
    ksuid = Ksuid()
    date_now = random.choice(rand_date).strftime("%Y-%m-%d %H:%M:%S")
    Item={
        'PK': 'COMMENT#' + str(ksuid),
        'SK': 'COMMENT#' + str(ksuid),
        'CreatedAt': date_now,
        'CommentingUser': random.choice(names),
        'Type': 'COMMENT',
        'GSI1PK': random.choice(pk),
        'GSI1SK': 'COMMENT#' + date_now,
    }
    all_items.append(Item)

In [75]:
# Save to json file
with open('fb.json', 'w') as outfile:
    json.dump(all_items, outfile, indent=4)

# Write to DynamoDB table
dt.load_json('fb.json')

Data loaded successfully from fb.json.


Ahora queremos añadir Comentarios. Los usuarios pueden comentar una publicación para dar ánimos o discutir sobre algún punto político pedante.

Con nuestra nueva entidad Comentario, tenemos un patrón de acceso relacional donde queremos obtener una publicación y los comentarios más recientes para esa publicación. ¿Cómo podemos modelar este nuevo patrón?

La colección de elementos Post ya se utiliza en la tabla base. Para manejar este patrón de acceso, necesitaremos crear una nueva colección de elementos en un índice secundario global.

In [25]:
dt.create_global_secondary_index(
    att_name="GSI1PK",
    att_type="S",
    sort_index="GSI1SK",
    sort_type="S",
    i_name="GSI1"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        time.sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...
Global secondary index created. Time elapsed: 12.06 minute


Para ello, vamos a añadir los siguientes atributos al elemento Post:
* GSI1PK: POST#<PostId>
* GSI1SK: POST#<PostId>

Y crearemos un elemento Comentario con los siguientes atributos:
* PK: COMMENT#<CommentId>
* SK: COMMENT#<CommentId>
* GSI1PK: POST#<PostId>
* GSI1SK: COMMENT#<Sello de tiempo>

Tendrá que ejecutar un escaneo de tabla en su tabla y actualizar cada uno de los elementos Post para añadir estos nuevos atributos.


In [76]:
dynamodb_client = boto3.client('dynamodb', region_name='us-east-1')

params = {
    "TableName": dt.table_name,
	"FilterExpression": "#3deb0 = :3deb0",
	"ExpressionAttributeNames": {"#3deb0":"Type"},
	"ExpressionAttributeValues": {":3deb0": {"S":"POST"}},
	"Limit": 100
}

In [79]:
total_items = 500
while total_items > 0:
    last_evaluated = dynamodb_client.scan(**params) 
    len_items = len(last_evaluated['Items'])
    if len_items == 0:
        print("No more items to scan.")
        break
    else:
        print("Scanned {0} items".format(len_items))
        for item in last_evaluated['Items']:
            try:
                dynamodb_client.update_item(
                    TableName=dt.table_name,
                    Key={
                        'PK': item['PK'],
                        'SK': item['SK']
                    },
                    UpdateExpression="set #3deb0 = :3deb0, #3deb1 = :3deb1, #3deb2 = :3deb2",
                    ExpressionAttributeNames={
                        "#3deb0": "GSI1PK",
                        "#3deb1": "GSI1SK",
                        "#3deb2": "Type"
                    },
                    ExpressionAttributeValues={
                        ":3deb0": item['PK'],
                        ":3deb1": item['SK'],
                        ":3deb2": {"S": "POST-v1"}
                    }
                )
            except ClientError as error:
                print(f"Something went wrong while updating item {item['PK']} - {item['SK']}")
                print(error.response['ResponseMetadata'])
            
        if last_evaluated.get('LastEvaluatedKey'):
            if params.get('ExclusiveStartKey') == last_evaluated.get('LastEvaluatedKey'):
                break
            params['ExclusiveStartKey'] = last_evaluated.get('LastEvaluatedKey')
    total_items -= len_items
    if total_items <= 0:
        print("Total items reached.")

Scanned 17 items
Scanned 17 items
Scanned 12 items
Scanned 19 items
Scanned 15 items
No more items to scan.


#### Obtener los 2 ultimos comentarios de una publicación

In [95]:
from pprint import pprint
response = dt.query(pk[0], index_name = 'GSI1', scan_index_forward=False, consumed_capacity='TOTAL')
for i in range(3):
    pprint(response['Items'][i])

Consumed Capacity: 0.5
{'GSI1PK': 'POST#2SCyiZNTmDCDQajYbNQTH1ubaTA',
 'GSI1SK': 'POST#2SCyiZNTmDCDQajYbNQTH1ubaTA',
 'PK': 'POST#2SCyiZNTmDCDQajYbNQTH1ubaTA',
 'PostContent': 'https://...',
 'PostData': '2021-08-10 03:01:25',
 'PostOwner': 'Alex DeBrie',
 'SK': 'POST#2SCyiZNTmDCDQajYbNQTH1ubaTA',
 'Type': 'POST-v1'}
{'CommentingUser': 'James Dean',
 'CreatedAt': '2023-11-18 12:05:16',
 'GSI1PK': 'POST#2SCyiZNTmDCDQajYbNQTH1ubaTA',
 'GSI1SK': 'COMMENT#2023-11-18 12:05:16',
 'PK': 'COMMENT#2SCyjBDRkCY7mNvyr1GtynoExzq',
 'SK': 'COMMENT#2SCyjBDRkCY7mNvyr1GtynoExzq',
 'Type': 'COMMENT'}
{'CommentingUser': 'Laura Dern',
 'CreatedAt': '2022-12-12 14:39:03',
 'GSI1PK': 'POST#2SCyiZNTmDCDQajYbNQTH1ubaTA',
 'GSI1SK': 'COMMENT#2022-12-12 14:39:03',
 'PK': 'COMMENT#2SCyjCBp15g7G5YPrADV9ztq3TU',
 'SK': 'COMMENT#2SCyjCBp15g7G5YPrADV9ztq3TU',
 'Type': 'COMMENT'}
Consumed capacity: 0.50


A medida que recibimos elementos del resultado de la exploración, iteramos sobre ellos y realizamos una solicitud de API UpdateItem para añadir las propiedades relevantes a los elementos existentes.

Hay algo de trabajo adicional para manejar el valor LastEvaluatedKey que se recibe en una respuesta de Escaneo. Esto indica si tenemos elementos adicionales para escanear o si hemos llegado al final de la tabla.

Hay algunas cosas que se pueden mejorar, como el uso de escaneos paralelos, la adición de tratamiento de errores y la actualización de varios elementos en una solicitud BatchWriteItem, pero esta es la forma general del proceso ETL. Hay una nota sobre los escaneos paralelos al final de este capítulo.

Esta es la parte más difícil de una migración, y querrá probar su código a fondo y supervisar el trabajo cuidadosamente para asegurarse de que todo va bien. Sin embargo, no hay mucho que hacer. Mucho de esto se puede parametrizar:
* ¿Cómo sé qué artículos quiero?
* Una vez que tengo mis artículos, ¿qué nuevos atributos tengo que añadir?

A partir de ahí, sólo tienes que tomarte el tiempo necesario para que se ejecute toda la operación de actualización.
